<a href="https://colab.research.google.com/github/SodiKroehler/MSTHESIS/blob/main/MSTHESIS_tinyllamaOnQbias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!ls

bert_crc.py	MSTHESIS_allSplits_tinyllamaOnQbias.ipynb
makeDataset.py	requirements.txt
models		week5_qbias_dataset.csv
models_llama


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [20]:
# !nohup python script.py > output.log 2>&1 & 
run_all_llamas()

Epoch 1, Loss: 0.7091535722828466
Epoch 2, Loss: 0.6977472050956871
Epoch 3, Loss: 0.69547550686602
Epoch 4, Loss: 0.6947181076035015


In [29]:
import os
get_ipython().system=os.system

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

In [ ]:
import torch
import os






In [9]:
# def evaluate_model(rdf, modelName, labelColumn):
rdf.dropna(subset=['raw'], inplace=True)
rdf = rdf[rdf['raw'].str.len() > 0]
modelName = "week6large_llama_binary_evenSplit_left"
labelColumn = "label_left"


save_dir = f"./models_llama/{modelName}"
if not os.path.exists(save_dir):
    logger.warn("Model not exists")
    # return


device = "cuda" if torch.cuda.is_available() else "cpu"

true_labels = []
pred_labels = []
label_encoder = None

with open(os.path.join(save_dir, "label_encoder.pkl"), "rb") as f:
    label_encoder = pickle.load(f)

rdf["class_id"] = label_encoder.fit_transform(rdf[labelColumn])

labels = torch.tensor(rdf["class_id"].values, dtype=torch.long)

dataset = MultiClassDataset(rdf["raw"].tolist(), labels, tokenizer)
dataloader = DataLoader(dataset, batch_size=3, shuffle=True)

num_classes = len(label_encoder.classes_)
print(label_encoder.classes_)

model = LlamaMultiClassClassifier("TinyLlama/TinyLlama-1.1B-Chat-v1.0", num_classes).to(device)
model.load_state_dict(torch.load(os.path.join(save_dir, "model.pth"), map_location=device))
model.eval()

with torch.no_grad():
    for batch in dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].cpu().numpy()

        logits = model(input_ids, attention_mask)
        print(logits)
        predictions = torch.argmax(logits, dim=1).cpu().numpy()

        true_labels.extend(labels)
        pred_labels.extend(predictions)

        break


true_labels = label_encoder.inverse_transform(true_labels)
pred_labels = label_encoder.inverse_transform(pred_labels)


print("Classification Report:")
print(classification_report(true_labels, pred_labels))
accuracy = accuracy_score(true_labels, pred_labels)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, pred_labels, average="weighted")
cm = confusion_matrix(true_labels, pred_labels)

    # return [modelName, accuracy, precision, recall, f1]


[0 1]
tensor([[-0.2689, -0.3379],
        [-0.2689, -0.3379],
        [-0.2689, -0.3379]], device='cuda:0')
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       0.0
           1       0.00      0.00      0.00       3.0

    accuracy                           0.00       3.0
   macro avg       0.00      0.00      0.00       3.0
weighted avg       0.00      0.00      0.00       3.0



/ihome/xli/sek188/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/ihome/xli/sek188/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/ihome/xli/sek188/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/ihome/xli/sek1

In [10]:
for name, param in model.named_parameters():
    print(f"{name}: requires_grad={param.requires_grad}, mean={param.data.mean():.4f}, std={param.data.std():.4f}")


llama.embed_tokens.weight: requires_grad=True, mean=-0.0000, std=0.0147
llama.layers.0.self_attn.q_proj.weight: requires_grad=True, mean=-0.0000, std=0.0162
llama.layers.0.self_attn.k_proj.weight: requires_grad=True, mean=-0.0001, std=0.0314
llama.layers.0.self_attn.v_proj.weight: requires_grad=True, mean=0.0000, std=0.0110
llama.layers.0.self_attn.o_proj.weight: requires_grad=True, mean=0.0000, std=0.0087
llama.layers.0.mlp.gate_proj.weight: requires_grad=True, mean=-0.0000, std=0.0165
llama.layers.0.mlp.up_proj.weight: requires_grad=True, mean=-0.0000, std=0.0168
llama.layers.0.mlp.down_proj.weight: requires_grad=True, mean=0.0000, std=0.0166
llama.layers.0.input_layernorm.weight: requires_grad=True, mean=0.0056, std=0.0449
llama.layers.0.post_attention_layernorm.weight: requires_grad=True, mean=0.0735, std=0.0326
llama.layers.1.self_attn.q_proj.weight: requires_grad=True, mean=0.0000, std=0.0290
llama.layers.1.self_attn.k_proj.weight: requires_grad=True, mean=-0.0000, std=0.0473
lla

In [ ]:
pred_labels

In [11]:
rdf = pd.read_csv("week5_qbias_dataset.csv")
rdf.dropna(subset=['raw'], inplace=True)
rdf = rdf[rdf['raw'].str.len() > 0]
rdf = rdf.sample(100, random_state=RANDOM_SEED+9)
evaluate_model(rdf, "large_llama_binary_evenSplit_left", "label_left")
# torch.cuda.empty_cache()

[0 1]
Classification Report:
              precision    recall  f1-score   support

           0       0.49      1.00      0.66        49
           1       0.00      0.00      0.00        51

    accuracy                           0.49       100
   macro avg       0.24      0.50      0.33       100
weighted avg       0.24      0.49      0.32       100



/ihome/xli/sek188/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/ihome/xli/sek188/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/ihome/xli/sek188/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


NameError: name 'predicted_labels' is not defined

In [17]:
res = pd.read_csv('week5_results.csv')
res.head()

,Model,Accuracy,Precision,Recall,F1,Start_Time,Duration,End_Date
0,small_llama_binary_left,0,0,0,0,1.741606e+09,-67.528759,2025-03-10 07:33:20
1,small_llama_binary_left,0,0,0,0,1.741607e+09,-63.983117,2025-03-10 07:41:15
2,small_llama_binary_left,0,0,0,0,1.741610e+09,-3119.657375,2025-03-10 09:30:01
